In [9]:
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en import English
import matplotlib.pyplot as plt
import time
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import itertools 
import collections
#from tensorflow import keras

In [3]:
asrs = pd.read_csv('./ASRS_data.csv', sep="|",nrows = 500)
abr1 = pd.read_csv('./ASRS_Abr.csv')

Preprocessing

In [4]:
abr1 = abr1[['Abr','word']]
abr1['Abr'] = abr1['Abr'].apply(lambda x : x.lower())
abr1['word'] = abr1['word'].apply(lambda x : x.lower())
trans_abr = abr1.set_index("Abr").T
abrd = trans_abr.to_dict('list')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """


In [5]:
def replace_abr(narr):
    # find all states that exist in the string
    abr_found = filter(lambda abr: abr in narr, abrd.keys())

    # replace each state with its abbreviation
    for abr in abr_found:
        narr = narr.replace(' '+abr+' ', ' '+abrd[abr][0]+' ')
    # return the modified string (or original if no states were found)
    return narr

In [6]:
asrs['Narrative'] = asrs['Narrative'].apply(lambda x : x.lower())
asrs["Narrative"] = asrs['Narrative'].str.replace('[^\w\s\d]','')
asrs["Narrative"] = asrs['Narrative'].str.replace(r'\d+','')
asrs['Narrative'] = asrs['Narrative'].apply(replace_abr)

In [8]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
clean = lambda new : " ".join([lemmatizer.lemmatize(i) for i in re.sub("[^a-zA-Z]", " " ,new.lower()).split() if i not in stop_words]).split()

In [10]:
asrs['cleaned']=asrs['Narrative'].apply(clean)

## Guided LDA

In [16]:
 import guidedlda as glda

In [17]:
#defining priors fron the research paper 13
lights = ['light', 'illuminated', 'caution', 'master', 'lights', 'panel', 
          'overhead','checklist', 'warning', 'maint']
passenger = ['flt', 'pax', 'attendants','attendant', 'seat', 'turbulence','fa'
            'seated', 'attendant','hit', 'cabin']
avoiding_ground = ['terrain', 'ground', 'gpws', 'warning', 'approach',
 'pull', 'climb', 'received', 'maneuver', 'approximately', 'atc']

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
docs = asrs['Narrative'].tolist()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)

In [19]:
word2id=vectorizer.vocabulary_
lights = [x for x in lights if x in list(word2id.keys())]
passenger = [x for x in passenger if x in list(word2id.keys())]
avoiding_ground = [x for x in avoiding_ground if x in list(word2id.keys())]
seed_topic_list = [lights,passenger,avoiding_ground]

In [20]:
model = glda.GuidedLDA(n_topics=5, n_iter=2000, random_state=7, refresh=20,alpha=0.01,eta=0.01)

seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id
        

In [21]:
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

INFO:guidedlda:n_documents: 500
INFO:guidedlda:vocab_size: 7154
INFO:guidedlda:n_words: 121786
INFO:guidedlda:n_topics: 5
INFO:guidedlda:n_iter: 2000


ValueError: expected sparse matrix with integer values, found float values